# 차원 축소
<hr>

**차원의 저주:** 차원이 커지면서 차원이 구성하는 공간의 크기가 커져 데이터들 간의 거리가 멀어지고 <br/>
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;그 결과, 데이터가 희박해지는 문제
<br/><br/>
데이터가 희박해지면 학습한 모형의 설명력이 떨어짐 <br/>
ex) 많은 양의 데이터 요구, KNN과 같은 거리에 기반한 알고리즘의 성능 저하

<br/>

## 차원의 저주 해결방법
1. 데이터 양 늘리기 <br/>
2. **차원 축소** <br/>
&nbsp; - 특성 선택: 대표 특성을 선택 -> 라쏘 회귀! <br/>
&nbsp; - 특성 추출(feature extraction): 기존 특성을 조합해 새로운 특성 생성 ex) PCA, LSA

In [4]:
# 1. PCA(Principal Component Analysis, 주성분 분석)
# 분산을 가장 크게 만드는 새로운 축을 찾는 방법론

# 20뉴스그룹 데이터 불러오기
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

# train
news_train = fetch_20newsgroups(subset='train',
                                remove=('headers', 'footers', 'quotes'),
                                categories=categories)

# test
news_test = fetch_20newsgroups(subset='test',
                               remove=('headers', 'footers', 'quotes'),
                               categories=categories)

In [6]:
# 전처리
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

cachedStopWords = stopwords.words('english')   # 불용어

from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

# train/test split
X_train = news_train.data
y_train = news_train.target

X_test = news_test.data
y_test = news_test.target

# 토큰화
reg = RegexpTokenizer("[\w']{3,}")
english_stops = set(stopwords.words('english'))

def tokenizer(text):
    tokens = reg.tokenize(text.lower())    # 텍스트를 소문자로 변환한 뒤 토큰화
    words = [word for word in tokens if word not in english_stops and len(word)>2]    # 불용어 제거
    
    # 포터 스테머
    
    # lambda로 어간추출한 것을 map을 이용해 words에 적용
    # 즉, words에서 어간추출
    features = (list(map(lambda token: PorterStemmer().stem(token), words)))
    
    return features

In [14]:
# tfidf
tfidf = TfidfVectorizer(tokenizer=tokenizer)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [15]:
# 차원 축소 전: 로지스틱 회귀분석
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train_tfidf, y_train)

print("# Train score: {:.3f}".format(lr.score(X_train_tfidf, y_train)))
print("# Test score: {:.3f}".format(lr.score(X_test_tfidf, y_test)))

# Train score: 0.962
# Test score: 0.761


In [16]:
### PCA
from sklearn.decomposition import PCA

# 차원 축소 전 차원의 크기
print("Original Tfidf matrix shape: ", X_train_tfidf.shape)

Original Tfidf matrix shape:  (2034, 20085)


In [17]:
# 차원 축소

# n_components: 축소할 차원의 크기
pca = PCA(n_components=2000, random_state=7)

X_train_pca = pca.fit_transform(X_train_tfidf.toarray())  # pca는 벡터에 대한 직접적 연산 지원하지 X, array로 변환 후 transform
X_test_pca = pca.transform(X_test_tfidf.toarray())

print("PCA Converted matrix shape: ", X_train_pca.shape)

# explained_variance_ratio: 새로운 분산을 축소 전 분산에 대한 비율로 나타냄, 즉 새로운 축의 설명력을 알 수 있음
print("Sum of explained variance ratio: {:.3f}".format(pca.explained_variance_ratio_.sum()))
# 설명되는 분산이 100%임을 알 수 있음

PCA Converted matrix shape:  (2034, 2000)
Sum of explained variance ratio: 1.000


In [18]:
# 축소된 데이터에 대해 로지스틱 회귀분석

lr.fit(X_train_pca, y_train)
print("# Train score: {:.3f}".format(lr.score(X_train_pca, y_train)))
print("# Test score: {:.3f}".format(lr.score(X_test_pca, y_test)))

# 1/10로 축소했음에도 똑같은 성능임을 알 수 있음

# Train score: 0.962
# Test score: 0.761


In [20]:
# 라쏘와 비교
lasso = LogisticRegression(penalty='l1', solver='liblinear', C=1)
lasso.fit(X_train_tfidf, y_train)

print("# Train score: {:.3f}".format(lasso.score(X_train_tfidf, y_train)))
print("# Test score: {:.3f}".format(lasso.score(X_test_tfidf, y_test)))

# 0이 아닌 계수 출력
import numpy as np

print("# Used features: {}".format(np.sum(lasso.coef_ != 0)), "out of ", X_train_tfidf.shape)

# Train score: 0.790
# Test score: 0.718
# Used features: 321 out of  (2034, 20085)


In [22]:
# 차원축소도 321개로

pca = PCA(n_components=321, random_state=7)

X_train_pca = pca.fit_transform(X_train_tfidf.toarray())  
X_test_pca = pca.transform(X_test_tfidf.toarray())

print("PCA Converted matrix shape: ", X_train_pca.shape)

print("Sum of explained variance ratio: {:.3f}".format(pca.explained_variance_ratio_.sum()))

lr.fit(X_train_pca, y_train)
print("# Train score: {:.3f}".format(lr.score(X_train_pca, y_train)))
print("# Test score: {:.3f}".format(lr.score(X_test_pca, y_test)))

# 특성의 수가 같을 때, 라쏘보다 성능이 좋음

PCA Converted matrix shape:  (2034, 321)
Sum of explained variance ratio: 0.437
# Train score: 0.875
# Test score: 0.751
